### Creating database

In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [11]:
#creating the database
def db_creation(password, db):
    
    cnx = mysql.connector.connect(user='root', password=f'{password}',
                                auth_plugin='mysql_native_password',
                                host='127.0.0.1')


    mycursor = cnx.cursor()
    try:
        mycursor.execute(f"CREATE DATABASE {db}")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [15]:
db_creation('AlumnaAdalab', 'HR_Database')

CMySQLCursor: CREATE DATABASE HR_Database


In [12]:
# opening the csv of the dataframe with df dtype and MySQL type 'traduction': 
col_types = pd.read_csv('data/HR_dataset/conversion_sql', index_col=0)

In [13]:
#Copyping the dictionaire of tables creation (table: columns)
col_table = {'working_profile': ['EmpID', 'Term', 'EmploymentStatus', 'DateofHire', 'DateofTermination', 'Salary', 'Department', 'Position', 'ManagerID', 'SpecialProjectsCount'],
            'personal_profile': ['EmpID', 'Employee_Name', 'DOB', 'MaritalStatusID','GenderID', 'FromDiversityJobFairID', 'CitizenDesc', 'RaceDesc', 'HispanicLatino', 'State', 'Zip'],
            'recruitment': ['EmpID', 'RecruitmentSource'],
            'performance_employee': ['EmpID', 'LastPerformanceReview_Date', 'PerformanceScore'],
            'engagement': ['EmpID', 'EngagementSurvey', 'EmpSatisfaction'],
            'absenteeism': ['EmpID', 'DaysLateLast30', 'Absences'],
            'attrition': ['EmpID', 'EmploymentStatus', 'DateofTermination', 'TermReason'],
            'manager': ['ManagerID', 'ManagerName'],
            'marital_status': ['MaritalStatusID', 'MaritalDesc', 'Married'],
            'gender': ['GenderID', 'Sex']}

In [77]:
columnas = col_table['working_profile']

formatted_line = "(" + ", ".join(columnas) + ")"

print(formatted_line)

(EmpID, Term, EmploymentStatus, DateofHire, DateofTermination, Salary, Department, Position, ManagerID, SpecialProjectsCount)


In [ ]:
for line in columnas:
    formatted_line = "(" + ", ".join(line) + ")"
    formatted_lines.append(formatted_line)

sql_insert_values = ",".join(formatted_lines)
sql_insert_values

In [18]:
#automating the tables creation:
def tables_creation(password, db, tables_dict):
    cnx = mysql.connector.connect(user='root', password=f'{password}',
                                host='127.0.0.1', database=f'{db}')

    mycursor = cnx.cursor()
    try:
        for table_name, columns in tables_dict.items():
            columns_sql = ', '.join([f'{column} {col_types.loc[col_types["index"] == column, "SQL_conv"].values[0]}' for column in columns])
            
            mycursor.execute(f'CREATE TABLE {table_name} ({columns_sql}, PRIMARY KEY ({columns[0]}));')
            print(mycursor)
            
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [19]:
tables_creation('AlumnaAdalab', 'HR_Database', col_table)

CMySQLCursor: CREATE TABLE working_profile (EmpID INT,..
CMySQLCursor: CREATE TABLE personal_profile (EmpID INT..
CMySQLCursor: CREATE TABLE recruitment (EmpID INT, Rec..
CMySQLCursor: CREATE TABLE performance_employee (EmpID..
CMySQLCursor: CREATE TABLE engagement (EmpID INT, Enga..
CMySQLCursor: CREATE TABLE absenteeism (EmpID INT, Day..
CMySQLCursor: CREATE TABLE attrition (EmpID INT, Emplo..
CMySQLCursor: CREATE TABLE manager (ManagerID INT, Man..
CMySQLCursor: CREATE TABLE marital_status (MaritalStat..
CMySQLCursor: CREATE TABLE gender (GenderID INT, Sex V..


In [22]:
#importing the tables to insert:
for k in col_table.keys():
    globals()[k] = pd.read_csv(f'data/HR_dataset/{k}.csv')

In [67]:
for table_name, columns in col_table.items():
    df = pd.read_csv(f'data/HR_dataset/{table_name}.csv')
    
    list_of_lists = [row.tolist() for index, row in df.iterrows()]

    formatted_lines = []

    for line in list_of_lists:
        formatted_line = "(" + ", ".join(map(str, line)) + ")"
        formatted_lines.append(formatted_line)

    sql_insert_values = ",".join(formatted_lines)

sql_insert_values

'(0, 1, M ),(2, 0, F)'

In [81]:
def insert_table(password, db, tables_dict):
    
    cnx = mysql.connector.connect(user='root', password=f'{password}',
                                host='127.0.0.1', database=f'{db}')

    mycursor = cnx.cursor()
    

    try: 
        
        for table_name, columns in tables_dict.items():
            
            df = pd.read_csv(f'data/HR_dataset/{table_name}.csv') # opening the dataframe
            
            list_of_lists = [row.tolist() for index, row in df.iterrows()] # creating a list of each row from each dataframe

            formatted_lines = []

            for line in list_of_lists:
                formatted_line = "(" + ", ".join(str(item) for item in line) + ")" # formatting each row-list for SQL input
                formatted_lines.append(formatted_line)

            sql_insert_values = ",".join(formatted_lines)
            

            
        print((f'INSERT INTO {table_name} VALUES {sql_insert_values};'))
        mycursor.execute(f'INSERT INTO {table_name} ({columns}) VALUES {sql_insert_values};')
        cnx.commit() 

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [82]:
insert_table('AlumnaAdalab', 'HR_Database', col_table)

INSERT INTO gender VALUES (0, 1, M ),(2, 0, F);
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '['GenderID', 'Sex']) VALUES (0, 1, M ),(2, 0, F)' at line 1
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '['GenderID', 'Sex']) VALUES (0, 1, M ),(2, 0, F)' at line 1


In [7]:
#creating foreign keys:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='HR_Database')

mycursor = cnx.cursor()
try:

    mycursor.execute("""ALTER TABLE `working_profile` 
                     ADD CONSTRAINT `fk_working_personal` FOREIGN KEY (`EmpID`) REFERENCES `personal_profile` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_recruitment` FOREIGN KEY (`EmpID`) REFERENCES `recruitment` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_performance_employee` FOREIGN KEY (`EmpID`) REFERENCES `performance_employee` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_engagement` FOREIGN KEY (`EmpID`) REFERENCES `engagement` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_absenteeism` FOREIGN KEY (`EmpID`) REFERENCES `absenteeism` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_attrition` FOREIGN KEY (`EmpID`) REFERENCES `attrition` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE; 
                     ALTER TABLE `manager` ADD CONSTRAINT `fk_manager_working` FOREIGN KEY (`ManagerID`) REFERENCES `working_profile` (`ManagerID`) ON DELETE RESTRICT ON UPDATE CASCADE; 
                     ALTER TABLE `marital_status` ADD CONSTRAINT `fk_marital_personal` FOREIGN KEY (`MaritalStatusID`) REFERENCES `personal_profile`(`MaritalStatusID`) ON DELETE RESTRICT ON UPDATE CASCADE; 
                     ALTER TABLE `gender` ADD CONSTRAINT `fk_gender_personal` FOREIGN KEY (`GenderID`) REFERENCES `personal_profile` (`GenderID`) ON DELETE RESTRICT ON UPDATE CASCADE;""")
    print(mycursor)
    
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: ALTER TABLE `working_profile` 
         ..
